# QUESTION: Job Applicant Selection

Input: \
Educational background - high school, diploma, degree, master, doctoral, postdoc \
Professional qualifications - inadequate, adequate, exceptional \
Working experience - very low, low, medium, high, very high

Output: \
Qualification - low, medium, high 

Rules:

IF (educational background is high school) AND (professional qualifications are inadequate OR working experience is very low) THEN qualification is low. \
IF (educational background is high school) AND (professional qualifications are adequate) AND (working experience is low) THEN qualification is low. \
IF (educational background is high school) AND (professional qualifications are exceptional) AND (working experience is medium) THEN qualification is medium. \
IF (educational background is high school) AND (professional qualifications are exceptional) AND (working experience is high OR very high) THEN qualification is high.

IF (educational background is diploma) AND (professional qualifications are inadequate OR working experience is very low) THEN qualification is low. \
IF (educational background is diploma) AND (professional qualifications are adequate OR exceptional) AND (working experience is low) THEN qualification is medium. \
IF (educational background is diploma) AND (professional qualifications are exceptional) AND (working experience is medium OR high OR very high) THEN qualification is high.

IF (educational background is degree OR higher) AND (professional qualifications are inadequate OR working experience is very low) THEN qualification is low. \
IF (educational background is degree OR higher) AND (professional qualifications are adequate) AND (working experience is low) THEN qualification is medium. \
IF (educational background is degree OR higher) AND (professional qualifications are exceptional) AND (working experience is medium OR high OR very high) THEN qualification is high.

Q: How qualified is the candidate if the educational background is rated 40 out of 100, the professional qualifications is rated 80 out of 100, and the years of working experience is 1?

## 1. Install scikit-fuzzy package & libraries

In [ ]:
# install fuzzy package
!pip install -U scikit-fuzzy

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

## 2. Declare fuzzy variable and membership function

In [ ]:
# new Antecedent/Consequent objects hold universe variables and membership functions

educational_background = ctrl.Antecedent(np.arange(0, 101, 1), 'educational background')
professional_qualification = ctrl.Antecedent(np.arange(0, 101, 1), 'professional qualification')
working_experience = ctrl.Antecedent(np.arange(0, 41, 1), 'working experience')

qualification = ctrl.Consequent(np.arange(0, 21, 1), 'qualification')

## 3. Create fuzzy membership function for Educational Background

In [ ]:
# create membership function for educational background
educational_background['high school'] = fuzz.trapmf(educational_background.universe, [0, 0, 10, 20])
educational_background['diploma'] = fuzz.trimf(educational_background.universe, [10, 20, 40])
educational_background['degree'] = fuzz.trimf(educational_background.universe, [20, 40, 60])
educational_background['master'] = fuzz.trimf(educational_background.universe, [40, 60, 80])
educational_background['doctoral'] = fuzz.trimf(educational_background.universe, [60, 80, 100])
educational_background['postdoc'] = fuzz.trapmf(educational_background.universe, [80, 100, 110, 111])

# visualize plot generated 
educational_background.view()

## 4. Create fuzzy membership function for Professional Qualification

In [ ]:
# create membership function for professional qualification
professional_qualification['inadequate'] = fuzz.trimf(professional_qualification.universe, [0, 0, 50])
professional_qualification['adequate'] = fuzz.trimf(professional_qualification.universe, [0, 50, 100])
professional_qualification['exceptional'] = fuzz.trimf(professional_qualification.universe, [50, 100, 150])

# visualize plot generated 
professional_qualification.view()

## 5. Create fuzzy membership function for Working Experience

In [ ]:
# custom membership function for Working Experience
working_experience['very low'] = fuzz.trimf(working_experience.universe, [0, 0, 10])
working_experience['low'] = fuzz.trimf(working_experience.universe, [0, 10, 20])
working_experience['medium'] = fuzz.trimf(working_experience.universe, [10, 20, 30])
working_experience['high'] = fuzz.trimf(working_experience.universe, [20, 30, 40])
working_experience['very high'] = fuzz.trimf(working_experience.universe, [30, 40, 50])
    
# visualize plot generated 
working_experience.view()

## 6. Create fuzzy membership function for Qualification

In [ ]:
# custom membership function for Qualification
qualification['low'] = fuzz.trimf(qualification.universe, [0, 0, 5])
qualification['medium'] = fuzz.trapmf(qualification.universe, [0, 5, 10, 15])
qualification['high'] = fuzz.trapmf(qualification.universe, [10, 15, 20, 21])

# visualize plot generated 
qualification.view()

## 7. Construct fuzzy rules

In [ ]:
# create rules
rule1 = ctrl.Rule(educational_background['high school'] & (professional_qualification['inadequate'] | working_experience['very low']), qualification['low'])
rule2 = ctrl.Rule(educational_background['high school'] & professional_qualification['adequate'] & working_experience['low'], qualification['low'])
rule3 = ctrl.Rule(educational_background['high school'] & professional_qualification['exceptional'] & working_experience['medium'], qualification['medium'])
rule4 = ctrl.Rule(educational_background['high school'] & professional_qualification['exceptional'] & (working_experience['high'] | working_experience['very high']), qualification['high'])

rule5 = ctrl.Rule(educational_background['diploma'] & (professional_qualification['inadequate'] | working_experience['very low']), qualification['low'])
rule6 = ctrl.Rule(educational_background['diploma'] & (professional_qualification['adequate'] | professional_qualification['exceptional']) & working_experience['low'], qualification['medium'])
rule7 = ctrl.Rule(educational_background['diploma'] & professional_qualification['exceptional'] & (working_experience['medium'] | working_experience['high'] | working_experience['very high']), qualification['high'])

rule8 = ctrl.Rule((educational_background['degree'] | educational_background['master'] | educational_background['doctoral'] | educational_background['postdoc']) & (professional_qualification['inadequate'] | working_experience['very low']), qualification['low'])
rule9 = ctrl.Rule((educational_background['degree'] | educational_background['master'] | educational_background['doctoral'] | educational_background['postdoc']) & professional_qualification['adequate'] & working_experience['low'], qualification['medium'])
rule10 = ctrl.Rule((educational_background['degree'] | educational_background['master'] | educational_background['doctoral'] | educational_background['postdoc']) & professional_qualification['exceptional'] & (working_experience['medium'] | working_experience['high'] | working_experience['very high']), qualification['high'])

candidate_qualification_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10])
candidate_qualification = ctrl.ControlSystemSimulation(candidate_qualification_ctrl)

## 8. Defuzzification

In [ ]:
candidate_qualification.input['educational background'] = 40
candidate_qualification.input['professional qualification'] = 80
candidate_qualification.input['working experience'] = 1
candidate_qualification.compute()

print("Candidate qualification:", round(candidate_qualification.output['qualification'], 2)) 
qualification.view(sim=candidate_qualification)